### Step 1: Import required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Step 2: Read training and test data from the disk

In [2]:
# 2.1: Read training data from disk using pandas
train_in = pd.read_csv("./resources/train_in.csv")
train_out = pd.read_csv("./resources/train_out.csv")

In [3]:
# 2.2: Read test data from disk using pandas
test_in = pd.read_csv("./resources/test_in.csv")
test_out = pd.read_csv("./resources/test_out.csv")

### Step 3: Transform data into matrices

In [4]:
# Convert data into matrix
training_data_input = np.array(train_in)
training_data_output = np.array(train_out)

test_data_input = np.array(test_in)
test_data_output = np.array(test_out)

### Step 4: Initialise weight matrix

In [5]:
# 4.1: With zeros
def initialise_weights_with_zeros(n_features, n_classes):
    # (+ 1) is for the bias term
    return np.zeros((n_features + 1, n_classes)) 

In [6]:
# 4.2: With given number
def initialise_weights_with_given_value(n_features, n_classes, value):
    # (+ 1) is for the bias term
    return np.full((n_features + 1, n_classes), value)

In [7]:
# 4.3: With random values
def initialise_weights_with_random_values(n_features, n_classes):
    # (+ 1) is for the bias term
    # (* 0.1) is used to keep the random numbers between -1 and 1, i.e. keep the values small
    return np.random.randn(n_features + 1, n_classes) * 0.1

In [31]:
n_features = training_data_input.shape[1]
n_classes = len(set(training_data_output[:, 0]))

w = initialise_weights_with_random_values(n_features, n_classes)

### Step 5: Add bias to input matrices

In [9]:
def add_bias_term(x):
    # create a column with 1
    bias = np.ones((x.shape[0], 1))
    # append and return column with 1 to dataset
    return np.hstack((x, bias))

In [10]:
x_train = add_bias_term(training_data_input)
x_test = add_bias_term(test_data_input)

### Step 6: Define perceptron algorithms

#### Basic Train Algorithm

When a mistake is made (i.e., when the predicted class $\hat{y}$ does not match the true class $y$), the weight vectors are updated as follows:

- **Decrease** the weights for the wrongly predicted class $\hat{y}$:

$$
\mathbf{w}_{\hat{y}} = \mathbf{w}_{\hat{y}} - \eta \mathbf{x}
$$

- **Increase** the weights for the correct class $y$:

$$
\mathbf{w}_y = \mathbf{w}_y + \eta \mathbf{x}
$$

where $\eta$ is the learning rate, which controls how much the weights are adjusted after each mistake.

In [11]:
# 6.1: Basic train algorithm 
def perceptron_basic_train(w, x, learning_rate, i, increase = None, decrease = None):
    if increase: 
        w[:, i] += learning_rate * x;
    elif decrease:
        w[:, i] -= learning_rate * x;

    return w

#### Gradient Descent Algorithm

In [43]:
# 6.2: Gradient descent train algorithm 
def perceptron_gradient_descent_train(w, x, learning_rate, real_output):
    # Dot product
    scores = np.dot(x, w)
    
    # Softmax
    probabilities = np.exp(scores) / np.sum(np.exp(scores))
    
    # Classes
    n_classes = len(scores) 
    
    for i in range(n_classes):
        indicator = 0
        # Indicator function
        if i + 1 == real_output:
            indicator = 1
        else:
            indicator = 0
        
        gradient = (probabilities[i] - indicator) * x
        
        w[:, i] -= learning_rate * gradient
    
    return w

#### Dot Product

In [12]:
def perceptron_predict(x, w):
    # compute dot product
    scores = np.dot(x, w)
    # return max value of scores matrix
    return np.argmax(scores)

#### Accuracy

In [13]:
def calculate_accuracy(right_prediction, wrong_prediction):
    return right_prediction / (right_prediction + wrong_prediction) * 100

#### Training

In [14]:
# Perceptron training: Basic
def perceptron_training_basic(w, training_data_input, training_data_output, learning_rate, training_algorithm):
    right_prediction = 0
    wrong_prediction = 0
    
    for index, row in enumerate(training_data_input):
        real_output = training_data_output[index, :][0]
        prediction_index = perceptron_predict(row, w)

        if real_output == prediction_index + 1:
            training_algorithm(w, row, learning_rate, prediction_index, increase = True)
            right_prediction += 1;
        else:
            training_algorithm(w, row, learning_rate, prediction_index, decrease = True)
            wrong_prediction += 1;
    
    accuracy = calculate_accuracy(right_prediction, wrong_prediction)

    return accuracy

In [52]:
# Perceptron training: Gradient Descent
def perceptron_training_gradient_descent(w, training_data_input, training_data_output, learning_rate, training_algorithm):
    right_prediction = 0
    wrong_prediction = 0
    
    for index, row in enumerate(training_data_input):
        real_output = training_data_output[index, :][0]
        prediction_index = perceptron_predict(row, w)
        
        training_algorithm(w, row, learning_rate, real_output)
        
        if real_output == prediction_index + 1:
            right_prediction += 1;
        else:
            wrong_prediction += 1;
    
    accuracy = calculate_accuracy(right_prediction, wrong_prediction)

    return accuracy

#### Epoch

In [22]:
# Basic
def run_n_epochs(n_epochs, learning_rate):
    accuracy = 0
    for i in range(n_epochs):
         # Change train algorithm
        acc = perceptron_training_basic(w, x_train, training_data_output, learning_rate, perceptron_basic_train)
        accuracy = acc;
    return accuracy

In [50]:
# Basic
def run_n_epochs(n_epochs, learning_rate):
    accuracy = 0
    for i in range(n_epochs):
         # Change train algorithm
        acc = perceptron_training_gradient_descent(w, x_train, training_data_output, learning_rate, perceptron_gradient_descent_train)
        accuracy = acc;
    return accuracy

### 7: Train

In [55]:
train_accuracy = run_n_epochs(100, 0.01)
print(f"Train Accuracy: {train_accuracy:.2f}%")

Train Accuracy: 14.77%


### 8: Test

In [28]:
def perceptron_test(w, testing_data_input, testing_data_output):
    right_prediction = 0
    wrong_prediction = 0
    
    for index, row in enumerate(testing_data_input):
        real_output = testing_data_output[index, :][0]
        prediction_index = perceptron_predict(row, w)

        if real_output == prediction_index + 1:
            right_prediction += 1;
        else:
            wrong_prediction += 1;
    
    accuracy = calculate_accuracy(right_prediction, wrong_prediction)

    return accuracy

In [37]:
test_accuracy = perceptron_test(w, x_test, test_data_output)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 41.64%


In [ ]:
print(accuracy)

In [ ]:
n_features = training_data_input.shape[1]
n_classes = len(set(training_data_output[:, 0]))

w = initialise_weights_with_random_values(n_features, n_classes)
training_data_input = add_bias_term(training_data_input)

perceptron_training(w, training_data_input, training_data_output, 0.001, perceptron_basic_train)

In [ ]:
accuracy = 0

In [ ]:
acc = 0
for i in range(2000):
    a = perceptron_training(w, training_data_input, training_data_output, 0.01, perceptron_basic_train)
    acc = a;
print(acc)

In [ ]:
x = np.full((1, 8), 1.0)
w = np.full((8, 10), 2.0)

print(x)
print(w)
basic_training(w,x, 0.5, 2, increase = True)
# w[:, 2] += x[0, :];
print(x)
print(w)

In [ ]:
print(perceptron_predict(np.full((2, 3), 1), np.full((3, 2), 2)))

In [ ]:
x = range(0, 10)
print(list(x))

In [ ]:
x = np.full((1, 8), 1)
w = np.full((8, 10), 2)
d = np.dot(x, w)
m = np.argmax(d, axis=1)
print(x, w, d)
print(m)

q = [[16, 16, 16, 16, 16, 16, 21, 16, 16, 16]]
m = np.argmax(q)
print(m)